# Conexão com o dynamoDB na AWS

#### importar boto3 e conectar o banco com as credenciais

In [33]:
import boto3
import tqdm

In [34]:
dynamodb = boto3.resource(service_name = 'dynamodb',region_name = 'sa-east-1',
              aws_access_key_id = '("SEU ACESS KEY ID")',
              aws_secret_access_key = '"SUA SECRET KEY"')

#### Criando a tabela MUSIC

In [35]:
table = dynamodb.create_table(
        TableName='Music',
        KeySchema=[
            {
                'AttributeName': 'Artist',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'SongTitle',
                'KeyType': 'RANGE'  # Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'Artist',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'SongTitle',
                'AttributeType': 'S'
            },
 
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 5
        }
    )        
print("Table status:", table.table_status)

Table status: CREATING


#### Importando a lib Json e carregando um arquivo com dados para inserir na tabela

In [36]:
import json
file = open(r"C:\Users\eider\Documents\AwsDynamoDB\src\itemmusic.json")
file = file.read()
data_file = json.loads(file)
data_file

{'Artist': 'Iron Maiden',
 'SongTitle': 'Chains of Misery',
 'AlbumTitle': 'Fear of the Dark',
 'SongYear': '1992'}

#### Conectando a tabela e verificando seu status no BD

In [37]:
music_table = dynamodb.Table('Music')
music_table.table_status

'ACTIVE'

#### Inserindo um registros na tabela

In [38]:
response = music_table.put_item(Item = data_file )

#### Inserindo multiplos registros na mesma tabela

In [39]:
file = open(r"C:\Users\eider\Documents\AwsDynamoDB\src\batchmusic.json")
file = file.read()
data_file = json.loads(file)
data_file

[{'Artist': 'Iron Maiden',
  'SongTitle': 'Wasting Love',
  'AlbumTitle': 'Fear of the Dark Live',
  'SongYear': '1992'},
 {'Artist': 'Iron Maiden',
  'SongTitle': 'Weekend Warrior',
  'AlbumTitle': 'Fear of the Dark',
  'SongYear': '1992'},
 {'Artist': 'Iron Maiden',
  'SongTitle': 'Fear of the Dark',
  'AlbumTitle': 'Fear of the Dark Tour',
  'SongYear': '1992'}]

In [40]:
with music_table.batch_writer() as batch:
    for record in tqdm.tqdm(data_file):
        batch.put_item(Item = record)

100%|██████████| 3/3 [00:00<00:00, 3012.43it/s]


#### Criar um index global secundário baseado no título do álbum.
#### O objetivo do index secundário é permitir realizar consultas diretas sem a necessidde do indice primário.

In [41]:
table = dynamodb.Table('Music') 
response = table.update(
    AttributeDefinitions=[
        {
            'AttributeName': 'AlbumTitle',
            'AttributeType': 'S'
        },
    ],
    GlobalSecondaryIndexUpdates=[
          {              
              'Create': {
                  'IndexName': 'AlbumTitle-index',
                  'KeySchema': [
                      {
                          'AttributeName': 'AlbumTitle',
                          'KeyType': 'HASH'
                      },
                  ],
                  'Projection': {
                      'ProjectionType': 'ALL'                      
                  },
                  'ProvisionedThroughput': {
                      'ReadCapacityUnits': 10,
                      'WriteCapacityUnits': 5
                  }
              },
              
          },
      ],
)

#### Criar um index global secundário baseado no nome do artista e no título do álbum.

In [43]:
table = dynamodb.Table('Music') 
response = table.update(
    AttributeDefinitions=[
        {
            'AttributeName': 'Artist',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'AlbumTitle',
            'AttributeType': 'S'
        },
    ],
    GlobalSecondaryIndexUpdates=[
          {              
              'Create': {
                  'IndexName': 'ArtistAlbumTitle-index',
                  'KeySchema': [
                      {
                          'AttributeName': 'Artist',
                          'KeyType': 'HASH'
                      },
                      {
                          'AttributeName': 'AlbumTitle',
                          'KeyType': 'RANGE'
                      },
                  ],
                  'Projection': {
                      'ProjectionType': 'ALL'                      
                  },
                  'ProvisionedThroughput': {
                      'ReadCapacityUnits': 10,
                      'WriteCapacityUnits': 5
                  }
              },
              
          },
      ],
)

#### Criar um index global secundário baseado notítulo da música e no ano.

In [44]:
table = dynamodb.Table('Music') 
response = table.update(
    AttributeDefinitions=[
        {
            'AttributeName': 'SongTitle',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'SongYear',
            'AttributeType': 'S'
        },
    ],
    GlobalSecondaryIndexUpdates=[
          {              
              'Create': {
                  'IndexName': 'SongTitleYear-index',
                  'KeySchema': [
                      {
                          'AttributeName': 'SongTitle',
                          'KeyType': 'HASH'
                      },
                      {
                          'AttributeName': 'SongYear',
                          'KeyType': 'RANGE'
                      },
                  ],
                  'Projection': {
                      'ProjectionType': 'ALL'                      
                  },
                  'ProvisionedThroughput': {
                      'ReadCapacityUnits': 10,
                      'WriteCapacityUnits': 5
                  }
              },

          },
      ],
)

#### Pesquisando por nome do artista e nome da música

In [45]:
from boto3.dynamodb.conditions import Key, Attr
dynamodb = boto3.resource('dynamodb', region_name='sa-east-1')
table = dynamodb.Table('Music')
response = table.scan(
    FilterExpression=Attr('Artist').eq("Iron Maiden") & Attr('SongTitle').eq('Weekend Warrior')
)
items = response['Items']
print(items)

[{'AlbumTitle': 'Fear of the Dark', 'Artist': 'Iron Maiden', 'SongYear': '1992', 'SongTitle': 'Weekend Warrior'}]


#### Outra maneira para a mesma consulta

In [46]:
response = table.get_item(
    Key={
        'Artist': 'Iron Maiden',
        'SongTitle': 'Weekend Warrior'
    }
)
item = response['Item']
print(item)

{'AlbumTitle': 'Fear of the Dark', 'Artist': 'Iron Maiden', 'SongYear': '1992', 'SongTitle': 'Weekend Warrior'}


## Consulta pelos indices secundários

In [47]:
file = open(r'C:\Users\eider\Documents\AwsDynamoDB\src/AlbumTitle.json')
file = file.read()
data_file = json.loads(file)
data_file

{':name': {'S': 'Fear of the Dark'}}

In [48]:
from boto3.dynamodb.conditions import Key, Attr
dynamodb = boto3.resource('dynamodb', region_name='sa-east-1')
table = dynamodb.Table('Music')
response = table.query(IndexName='AlbumTitle-index',KeyConditionExpression=Key('AlbumTitle').eq('Fear of the Dark'))
items = response['Items']
print(items)

[{'AlbumTitle': 'Fear of the Dark', 'Artist': 'Iron Maiden', 'SongYear': '1992', 'SongTitle': 'Weekend Warrior'}, {'AlbumTitle': 'Fear of the Dark', 'Artist': 'Iron Maiden', 'SongYear': '1992', 'SongTitle': 'Chains of Misery'}]


In [49]:
file = open(r'C:\Users\eider\Documents\AwsDynamoDB\src/songtitle_year.json')
file = file.read()
data_file = json.loads(file)
data_file

{':v_song': {'S': 'Wasting Love'}, ':v_year': {'S': '1992'}}

In [50]:
from boto3.dynamodb.conditions import Key, Attr
dynamodb = boto3.resource('dynamodb', region_name='sa-east-1')
table = dynamodb.Table('Music')
response = table.query(IndexName='SongTitleYear-index',KeyConditionExpression=
                       Key('SongTitle').eq('Wasting Love')&
                       Key('SongYear').eq('1992'))
items = response['Items']
print(items)

[{'AlbumTitle': 'Fear of the Dark Live', 'Artist': 'Iron Maiden', 'SongTitle': 'Wasting Love', 'SongYear': '1992'}]
